Data Science Project: **Stock Market Sentiment Analyis**

**Group Members**  

---

1. Aditya Yadav
2. Akshay Wagh
3. Tanmay Yelwande

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


reading data

In [62]:
import pandas as pd

file_path = "/content/drive/MyDrive/DS - Stock Market Sentiment Analysis/stockmarket_sentiment.csv"
df = pd.read_csv(file_path)

In [66]:
df

,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1
...,...,...
5786,Industry body CII said #discoms are likely to ...,-1
5787,"#Gold prices slip below Rs 46,000 as #investor...",-1
5788,Workers at Bajaj Auto have agreed to a 10% wag...,1
5789,"#Sharemarket LIVE: Sensex off day’s high, up 6...",1


In [67]:
df.head()

,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1


In [68]:
df.tail()

,Text,Sentiment
5786,Industry body CII said #discoms are likely to ...,-1
5787,"#Gold prices slip below Rs 46,000 as #investor...",-1
5788,Workers at Bajaj Auto have agreed to a 10% wag...,1
5789,"#Sharemarket LIVE: Sensex off day’s high, up 6...",1
5790,"#Sensex, #Nifty climb off day's highs, still u...",1


In [69]:
df.describe()

,Sentiment
count,5791.000000
mean,0.272664
std,0.962192
min,-1.000000
25%,-1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5791 entries, 0 to 5790
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       5791 non-null   object
 1   Sentiment  5791 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 90.6+ KB


cleaning and preprocessing

In [71]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [72]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Load dataset
file_path = "/content/drive/MyDrive/DS - Stock Market Sentiment Analysis/stockmarket_sentiment.csv"
df = pd.read_csv(file_path)


print("Dataset preview:")
print(df.head())


df.dropna(inplace=True)


df.drop_duplicates(inplace=True)


lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))


text_column = "Text"


def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)


df['cleaned_text'] = df[text_column].apply(preprocess_text)


print("Cleaned Data Preview:")
print(df[[text_column, 'cleaned_text']].head())

# Save the cleaned dataset
df.to_csv("/content/drive/MyDrive/cleaned_stockmarket_sentiment.csv", index=False)
print("Cleaned dataset saved as cleaned_stockmarket_sentiment.csv")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Dataset preview:
                                                Text  Sentiment
0  Kickers on my watchlist XIDE TIT SOQ PNK CPW B...          1
1  user: AAP MOVIE. 55% return for the FEA/GEED i...          1
2  user I'd be afraid to short AMZN - they are lo...          1
3                                  MNTA Over 12.00            1
4                                   OI  Over 21.37            1
Cleaned Data Preview:
                                                Text  \
0  Kickers on my watchlist XIDE TIT SOQ PNK CPW B...   
1  user: AAP MOVIE. 55% return for the FEA/GEED i...   
2  user I'd be afraid to short AMZN - they are lo...   
3                                  MNTA Over 12.00     
4                                   OI  Over 21.37     

                                        cleaned_text  
0  kicker watchlist xide tit soq pnk cpw bpz aj t...  
1  user aap movie return feageed indicator trade ...  
2  user id afraid short amzn looking like nearmon...  
3                   

sentiment encoding and splitting

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Check unique sentiment values
print("Unique sentiment labels:", df["Sentiment"].unique())

# Ensure 'Sentiment' is numerical (if not, convert categorical labels)
df["Sentiment"] = df["Sentiment"].astype(int)

# Define features (X) and target (y)
X = df["cleaned_text"]
y = df["Sentiment"]

# Split into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert text data to numerical format using TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=5000)  # Use 5000 most important words
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("TF-IDF transformation complete. Training and testing data prepared.")

Unique sentiment labels: [ 1 -1]
TF-IDF transformation complete. Training and testing data prepared.


model training

In [74]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

model = LogisticRegression(max_iter=1000)

model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

# Print detailed classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Model Accuracy: 0.7800
Classification Report:
              precision    recall  f1-score   support

          -1       0.79      0.53      0.64       421
           1       0.78      0.92      0.84       738

    accuracy                           0.78      1159
   macro avg       0.78      0.73      0.74      1159
weighted avg       0.78      0.78      0.77      1159



In [75]:
import joblib

model_filename = "/content/drive/MyDrive/DS - Stock Market Sentiment Analysis/stock_sentiment_model.pkl"
vectorizer_filename = "/content/drive/MyDrive/DS - Stock Market Sentiment Analysis/tfidf_vectorizer.pkl"

joblib.dump(model, model_filename)
print(f"Model saved as: {model_filename}")

joblib.dump(vectorizer, vectorizer_filename)
print(f"TF-IDF Vectorizer saved as: {vectorizer_filename}")

Model saved as: /content/drive/MyDrive/DS - Stock Market Sentiment Analysis/stock_sentiment_model.pkl
TF-IDF Vectorizer saved as: /content/drive/MyDrive/DS - Stock Market Sentiment Analysis/tfidf_vectorizer.pkl
